# **SqueezeNet Architucture**

## Important Imports

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import os
import torchvision.transforms as transforms
from PIL import Image
from torchvision.transforms import functional as F
from torch.utils.data import Dataset, DataLoader
import torch
import torchvision
from torchvision import transforms
import pytorch_lightning as pl
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import Trainer
import torch.nn as nn
import torch.optim as optim
from pytorch_lightning.loggers import WandbLogger
import wandb
from collections import defaultdict
from tqdm import tqdm
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.io import read_image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pytorch_lightning as pl
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, EarlyStopping
from torchvision import transforms
from PIL import Image

In [2]:
path_train_images = '/kaggle/working/preprocessed-png'
train_ids = next(os.walk(path_train_images))[2]
img_list = os.listdir('/kaggle/working/preprocessed-png')
train_csv_data = pd.read_csv("/kaggle/input/mayo-clinic-strip-ai/train.csv")


In [3]:
# function to check if the image contains useful information after the crop.
def get_score(img):
    imgray = cv2.cvtColor(np.asarray(img), cv2.COLOR_RGB2GRAY)
    ret, thresh = cv2.threshold(imgray, 127, 255, 0)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    #print(len(hierarchy[0]), len(contours))
    return len(contours)

In [4]:
low_sc_images = []
def load_train_df():
    train_df = defaultdict(list)
    img_indx = 0
    print('Loading train images...')
    for i, proc_image_id in tqdm(enumerate(train_ids), total=len(train_ids)):
        fnames = ['0ba49d_0', '0ba49d_0', '006388_0']   
        if any(f in proc_image_id for f in fnames):
            continue
        if(train_csv_data.loc[train_csv_data['image_id'] == proc_image_id[:-8]].empty):
            continue
            
        label = train_csv_data.loc[train_csv_data['image_id'] == proc_image_id[:-8]]['label'].item()
        center_id = train_csv_data.loc[train_csv_data['image_id'] == proc_image_id[:-8]]['center_id'].item()
        
        path_tiles = path_train_images + proc_image_id
        train_df['image_id'].append(proc_image_id)
        train_df['label'].append(label)
        img_indx += 1
                
    return pd.DataFrame(train_df)

In [5]:
train_csv_data = pd.read_csv("/kaggle/input/mayo-clinic-strip-ai/train.csv")
train_df = load_train_df()


Loading train images...


100%|██████████| 4323/4323 [00:06<00:00, 685.74it/s]


## Needed Configurations

## Model Dataloaders

In [13]:

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


class BloodClotDataset(Dataset):
    def __init__(self, dataframe, image_dir, transform=None):
        self.dataframe = dataframe
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dir, self.dataframe.iloc[idx, 0])
        image = Image.open(img_name).convert("RGB")

        label = torch.tensor(self.dataframe.iloc[idx, 1])

        if self.transform:
            image = self.transform(image)

        return image, label
    
class BloodDataModule(pl.LightningDataModule):
    def __init__(self, train_dataframe, image_dir, batch_size=16, seed=19):
        super(BloodDataModule, self).__init__()
        self.train_dataframe = train_dataframe
        self.image_dir = image_dir
        self.batch_size = batch_size
        self.seed = seed
        self.label_mapping = {}

    def setup(self, stage=None):
        train_data, val_data = train_test_split(self.train_dataframe, test_size=0.2, random_state=self.seed)

        self.label_mapping = {label: idx for idx, label in enumerate(train_data['label'].unique())}

        train_data['label'] = train_data['label'].map(self.label_mapping)
        val_data['label'] = val_data['label'].map(self.label_mapping)

        self.train_dataset = BloodClotDataset(train_data, self.image_dir, transform=self.transform("train"))
        self.val_dataset = BloodClotDataset(val_data, self.image_dir, transform=self.transform("val"))

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=4)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, shuffle=False, num_workers=4)


    def transform(self, split):
        if split == "train":
            return transforms.Compose([
                transforms.RandomAffine(0, scale=(0.8, 1.2), translate=(0.1, 0.1)),
                transforms.RandomRotation(20),
                transforms.ToTensor(),
            ])
        elif split == "val":
            return transforms.Compose([
                transforms.RandomRotation(20),

                transforms.ToTensor(),

            ])

In [14]:
IMG_SIZE = 224

## Model Implemnetation

In [41]:
class Fire(nn.Module):
    def __init__(self, inplanes: int, squeeze_planes: int, expand1x1_planes: int, expand3x3_planes: int) -> None:
        super().__init__()
        self.inplanes = inplanes
        self.squeeze = nn.Conv2d(inplanes, squeeze_planes, kernel_size=1)
        self.squeeze_activation = nn.ReLU(inplace=True)
        self.squeeze_normalization = nn.BatchNorm2d(num_features=squeeze_planes)
        
        
        self.expand1x1 = nn.Conv2d(squeeze_planes, expand1x1_planes, kernel_size=1)
        self.expand3x3 = nn.Conv2d(squeeze_planes, expand3x3_planes, kernel_size=3, padding=1)
        self.expand_activation = nn.ReLU(inplace=True)
        
        self.expand_normalization = nn.BatchNorm2d(num_features=expand1x1_planes + expand3x3_planes)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.squeeze(x)
        x = self.squeeze_normalization(x)
        x = self.squeeze_activation(x)
        x =  torch.cat([self.expand1x1(x),self.expand3x3(x)],1)
        x = self.expand_normalization(x)
        x = self.expand_activation(x)
        
        return x

class SqueezeNet(pl.LightningModule):
    def __init__(self):
        super(SqueezeNet, self).__init__()

        self.conv0 = nn.Conv2d(3, 96, kernel_size=5, stride=2)
        self.bn0 = nn.BatchNorm2d(96)
        self.relu0 = nn.ReLU(inplace=True)
        
        self.maxpool0 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1,)
        
        self.fire1 =  Fire(96, 16, 64, 64)
        self.fire2 =  Fire(128, 16, 64, 64)
        self.fire3 =   Fire(128, 32, 128, 128)
        self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2,  padding=1)
        
        self.fire4 =  Fire(256, 32, 128, 128)
        self.fire5 =  Fire(256, 48, 192, 192)
        self.fire6 =   Fire(384, 48, 192, 192)
        self.fire7 =   Fire(384, 64, 256, 256)
        self.maxpool2 = nn.MaxPool2d(kernel_size=3, stride=2,  padding=1)
        self.fire8 = Fire(512, 64, 512, 512)
        self.drop = nn.Dropout(0.5)
        
        self.conv1 = nn.Conv2d(1024,2048, kernel_size=1)
        self.bn1 = nn.BatchNorm2d(2048)
        self.relu1 = nn.ReLU(inplace=True)
        self.avgpool =nn.AvgPool3d(kernel_size=(1024,14, 14))
        self.flatten = nn.Flatten()
        self.softmax = nn.Softmax(dim=1)

        


    def forward(self, x):
        x = self.conv0(x)
        x = self.bn0(x)
        x = self.relu0(x)
        x = self.maxpool0(x)
        x = self.fire1(x)
        x = self.fire2(x)
        x = self.fire3(x)
        x = self.maxpool1(x)
        x = self.fire4(x)
        x = self.fire5(x)
        x = self.fire6(x)
        x = self.fire7(x)
        x = self.maxpool2(x)
        x = self.drop(x)
        x = self.fire8(x)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x) 
        x = self.avgpool(x)        
        x = self.flatten(x)
        x = self.softmax(x)

        
        return x


    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = nn.CrossEntropyLoss(reduction="none")(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = torch.sum(preds == y).item() / len(y)
        self.log("train_loss",loss.mean(),on_epoch = True,prog_bar = True, on_step = True)
        self.log("train_acc", acc,on_epoch = True,prog_bar = True, on_step = True)
        return loss.mean()
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = nn.CrossEntropyLoss(reduction="none")(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = torch.sum(preds == y).item() / len(y)
        self.log("val_loss",loss.mean(),on_epoch = True,prog_bar = True, on_step = True)
        self.log("val_acc",acc,on_epoch = True,prog_bar = True, on_step = True)
        return {"val_loss": loss.mean(),"val_acc": acc}
      return {"avg_train_loss": avg_loss, "avg_train_acc": avg_acc}

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=0.0001)
        lr_scheduler = {'scheduler': optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=3,min_lr=1e-6),
                'monitor': 'train_loss'}

        return [optimizer], [lr_scheduler]


In [42]:
from torchinfo import summary

model = SqueezeNet()
batch_size = 16
summary(model, input_size=(1, 3, 224, 224))


Layer (type:depth-idx)                   Output Shape              Param #
SqueezeNet                               [1, 2]                    --
├─Conv2d: 1-1                            [1, 96, 110, 110]         7,296
├─BatchNorm2d: 1-2                       [1, 96, 110, 110]         192
├─ReLU: 1-3                              [1, 96, 110, 110]         --
├─MaxPool2d: 1-4                         [1, 96, 55, 55]           --
├─Fire: 1-5                              [1, 128, 55, 55]          --
│    └─Conv2d: 2-1                       [1, 16, 55, 55]           1,552
│    └─BatchNorm2d: 2-2                  [1, 16, 55, 55]           32
│    └─ReLU: 2-3                         [1, 16, 55, 55]           --
│    └─Conv2d: 2-4                       [1, 64, 55, 55]           1,088
│    └─Conv2d: 2-5                       [1, 64, 55, 55]           9,280
│    └─BatchNorm2d: 2-6                  [1, 128, 55, 55]          256
│    └─ReLU: 2-7                         [1, 128, 55, 55]          --
├

## Train the model

In [46]:
data_module = BloodDataModule(train_df, path_train_images, batch_size=16, seed=19)
model = SqueezeNet()

wandb_logger = WandbLogger(project="SqueezeNet-blood-clot",log_model="all")
# wandb_logger.experiment.config["batch_size"] = cfg.model.batch_size


checkpoint_loss = ModelCheckpoint(
    dirpath = "/kaggle/working//models",
    monitor="val_loss",
    filename="SqueezeNet-aug-loss-{val_loss:.2f}",
    save_top_k=1,
)

checkpoint_acc = ModelCheckpoint(
    dirpath = "/kaggle/working//models",
    monitor="val_acc",
    filename="SqueezeNet-aug-acc-{val_acc:.2f}",
    save_top_k=1,
)

lr_monitor = LearningRateMonitor(logging_interval='epoch')
# early_stopping = EarlyStopping(monitor='val_loss', min_delta=1e-5, patience=5, verbose=True, mode='min')

trainer = Trainer(
    logger = wandb_logger,
    accelerator="auto",
    callbacks=[lr_monitor, checkpoint_loss, checkpoint_acc],
    max_epochs = 200
) 

 
trainer.fit(model, datamodule=data_module)
wandb.finish()


/opt/conda/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:639: Checkpoint directory /kaggle/working//models exists and is not empty.


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
lr-Adam,██████████████████████▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc_epoch,▁▂▂▃▃▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇▇▇██████████████████
train_acc_step,▄▃▁▃▃▂▂▂▆▆▂▅▃▆▅▆▆▅▇█▆▆██▇▇██▇▇▇▇▇▇█▆████
train_loss_epoch,██▇▇▆▆▆▆▅▅▅▄▄▄▃▃▃▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss_step,▇██▆▆▆▇▆▄▅▇▅▅▄▅▃▄▄▂▂▄▃▁▁▂▂▂▁▃▂▃▂▁▂▁▃▁▃▁▂
trainer/global_step,▁▁▁▁▁▁▁▁▁▁▁▁▂▂▄▂▂▂▂▄▂▂▂▂▅▂▂▂▂▂▂▂▂▇▃▃▃▃█▃
val_acc_epoch,▁▂▂▃▃▄▄▄▂▄▅▄▆▅▆▅▄▆▅▅▆▆▇▇▇▇██▇██▇▇▇█▇███▇
val_acc_step,▆▃▂▂▆▅▂▅▁▆▆▇▇▆▃▆▇▆▅▆▆▅▆▇██▅▇███▂▆▇▇▅▆██▆
val_loss_epoch,█▇▇▇▇▆▆▆▇▅▄▅▄▄▄▄▄▃▄▃▃▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss_step,▆▆█▇▅▅▆▅█▅▅▃▄▃▅▄▃▃▅▅▂▄▂▂▂▂▄▁▁▂▂▄▃▂▃▃▃▂▁▄


In [45]:
wandb.finish()
